In [3]:
%pip install torch
%pip install transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 MB 2.6 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 3.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 3.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 3.1 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explici

In [5]:
import torch
from transformers import BertTokenizer, BertLMHeadModel

# Load the pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertLMHeadModel.from_pretrained(model_name)

# Fine-tune the model on your resume dataset
# Prepare your training data with input-output pairs: (input: skills, experiences; output: resume text)
# Split your data into training and validation sets

# Tokenize the input information
def tokenize_input(input_text):
    encoded_inputs = tokenizer.encode_plus(
        input_text,
        add_special_tokens=True,
        max_length=512,  # Adjust the maximum sequence length as needed
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )
    return encoded_inputs['input_ids'], encoded_inputs['attention_mask']

# Train the model
def train_model(train_inputs, train_masks, train_labels):
    model.train()
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
    loss_fn = torch.nn.CrossEntropyLoss()
    num_epochs = 10
    for epoch in range(num_epochs):
        optimizer.zero_grad()
        outputs = model(input_ids=train_inputs, attention_mask=train_masks, labels=train_labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

# Generate resumes
def generate_resume(input_text):
    model.eval()
    input_ids, attention_mask = tokenize_input(input_text)
    outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=512)
    generated_resume = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_resume

# Example usage



If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertLMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertLMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
Input length of input_ids is 512, but `max_length` is set to 512. This can lead to unexpected behavior. You should co

input information such as skills and experiences.


In [11]:
%pip install -U scikit-learn

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 5.2 MB/s eta 0:00:0000:0100:01
  Using cached threadpoolctl-3.1.0-py3-none-any.whl (14 kB)
Note: you may need to restart the kernel to use updated packages.


In [13]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Load the data from the CSV file
data = pd.read_csv("Resume.csv")
df = pd.DataFrame(data, columns=["Resume_str", "Category"])

# Prepare the input and label data
train_inputs = df['Category'].tolist()  # List of input texts
train_labels = df['Resume_str'].tolist()  # List of corresponding output texts

# Split your data into training and validation sets
train_ratio = 0.8  # 80% for training, 20% for validation
train_size = int(train_ratio * len(train_inputs))

train_inputs = train_inputs[:train_size]
train_labels = train_labels[:train_size]
val_inputs = train_inputs[train_size:]
val_labels = train_labels[train_size:]

# Encode labels using LabelEncoder
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
val_labels_encoded = label_encoder.transform(val_labels)

# Tokenize the training and validation data
def tokenize_inputs(input_texts):
    if isinstance(input_texts, list):
        encoded_inputs = tokenizer.encode_plus(
            input_texts,
            add_special_tokens=True,
            max_length=512,
            padding='max_length',
            truncation=True,
            return_tensors='pt',
            is_split_into_words=False
        )
    else:
        encoded_inputs = tokenizer.encode_plus(
            input_texts,
            add_special_tokens=True,
            max_length=512,
            padding='max_length',
            truncation=True,
            return_tensors='pt',
        )
    return encoded_inputs['input_ids'], encoded_inputs['attention_mask']

train_inputs_ids, train_attention_mask = tokenize_inputs(train_inputs)
val_input_ids, val_attention_mask = tokenize_inputs(val_inputs)

# Convert labels to tensors
train_labels = torch.tensor(train_labels_encoded)
val_labels = torch.tensor(val_labels_encoded)

# Train the model
train_model(train_inputs_ids, train_attention_mask, train_labels)


ValueError: Input [] is not valid. Should be a string, a list/tuple of strings or a list/tuple of integers.